In [6]:
# general
import os
import time
import random
import warnings
from threading import Thread # you can use easier threading packages
import sqlite3 as sqlite3
import csv
from csv import writer
import shutil
# ml
import numpy as np
import scipy as sp
import pandas as pd

# visual
import seaborn as sns
import matplotlib.pyplot as plt

# notebook
from IPython.display import display
import os
%config IPCompleter.greedy=True

In [7]:
input_data = []
for i in range(1,21):   
    firstname = ['John', 'Dana', 'Scott', 'Marc', 'Steven', 'Michael', 'Albert', 'Johanna']
    city = ['NewYork', 'Haifa', 'Munchen', 'London', 'PaloAlto',  'TelAviv', 'Kiel', 'Hamburg']
    secondname = ['Kelly','Brenda','Brendon','Steve','Jonah','Amy','Alex','Jerry']
    
    my_firstname= []
    my_city = []
    my_secondname = []
    
    for x in range(10):
        my_firstname.append(random.choice(firstname))
        my_city.append(random.choice(city))
        my_secondname.append(random.choice(secondname))
    
    myCSV = pd.DataFrame(list(zip(my_firstname,my_secondname,my_city)), columns =['firstname','secondname','city'])
    myCSV.to_csv('myCSV%s.csv' %i,encoding='utf-8',index = False)
    input_data.append('myCSV%s.csv' %i)  

In [8]:
def createfolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ("Error creating directory" + directory)
createfolder('./mapreducetemp/')

temp_results = pd.DataFrame(columns =['key','value'])
temp_results.to_csv('temp_results.csv',encoding='utf-8',index = False)
csvs_map = []
csvs_reduce = []
conn = sqlite3.connect('temp_results')
c = conn.cursor()
c.execute('CREATE TABLE IF NOT EXISTS temp_results (key text, value text)')
conn.commit()
createfolder('./mapreducetemp/')
createfolder('./mapreducefinal/')

In [9]:
def chunks(filename,chunk=16000):
    size=os.path.getsize(filename)
    part = int(size/chunk)
    size_left=size           
    fp = open(filename, 'rb')
    chunk_rows = []
    while (size_left>0):
        if (size_left<chunk):
            current_chunk = size_left
        else : 
            current_chunk = chunk
        fp.seek(size-size_left+1, 0)
        d2=fp.read(current_chunk).decode("utf-8")
        rows=d2.count("\n")
        size_left-=current_chunk
        chunk_rows.append(rows)
    return(chunk_rows)


def map_function(filename,x):
    print(f'Starting task: {x}...')
    data = pd.read_csv(filename)
    f= open('mapreducetemp/part-tmp-%s.csv'%x, 'w', encoding='UTF8',newline='') 
    csvs_map.append('mapreducetemp/part-tmp-%s.csv'%x)
    writer = csv.writer(f)
    header=["key","value"]
    writer.writerow(header)
    keyval=['0','0']
    for j in range (data.shape[0]):
        for line in (data.iloc[j,:]):
            words= line.split()
            for word in words:
                keyval[0]=word
                keyval[1]=filename
                writer.writerow(keyval)
    print(f'Finished task: {x}...')
    j+=1  
    return f.name

def batch(input_data,threshold=64000):
    size = 0
    x = 1
    batches = []
    all_data = pd.DataFrame()
    data = pd.DataFrame()
    for filename in input_data:
        if os.path.getsize(filename) > threshold:
            s = os.path.getsize(filename)
            all_chunks = chunks(filename,threshold)
            first = 0
            for chunk in all_chunks:      
                data = pd.read_csv(filename).iloc[first:chunk]
                f=open('mapreducetemp/batch-tmp-%s.csv'%x, 'w', encoding='UTF8',newline='')
                batches.append('mapreducetemp/batch-tmp-%s.csv'%x)
                data.to_csv(f,header=False,index=False)
                first+=chunk
                x+=1
        elif size + os.path.getsize(filename) <= threshold:
            all_data=all_data.append(data)
            data = pd.read_csv(filename)
            all_data=all_data.append(data.iloc[1:])
            size += os.path.getsize(filename)
        elif size + os.path.getsize(filename) > threshold:
            f= open('mapreducetemp/batch-tmp-%s.csv'%x, 'w', encoding='UTF8',newline='') 
            all_data.to_csv(f,header=False,index=False)
            batches.append('mapreducetemp/batch-tmp-%s.csv'%x)
            x+=1
            data = pd.read_csv(filename)
            f= open('mapreducetemp/batch-tmp-%s.csv'%x, 'w', encoding='UTF8',newline='')       
            size = os.path.getsize(filename)
    if batches == []:
        f= open('mapreducetemp/batch-tmp-%s.csv'%x, 'w', encoding='UTF8',newline='')
        all_data.to_csv(f,header=False,index=False)
        batches.append('mapreducetemp/batch-tmp-%s.csv'%x)
    print(batches)
    return batches


def thread_map_func(input_data ,Function=map_function,threshold=64000):
    # create threads
    threads = []
    x = 1
    batches = batch(input_data,threshold=threshold)
    for filename in batches:
        thread = Thread(target=Function, args=(filename,x))
        threads.append(thread)
        thread.start()
        x+=1

    # wait for the threads to complete
    for thread in threads:
        thread.join()
    print("finished mapping")
    
#load all csv's to the temp table
# def Temptable(filename):
def load_data(csvs_map):
    conn = sqlite3.connect('temp_results')
    c = conn.cursor()
    for file in csvs_map:
            df = pd.read_csv(file)
            df.to_sql('temp_results', conn, if_exists='append', index = False)
    c.execute('''SELECT  key,group_concat(value) 
         FROM temp_results
         GROUP BY 1
         ORDER BY 2 
                 ''')
    df1 = pd.DataFrame(c.fetchall(),columns=['key','value'])    
    return df1

def reduce_function(key,value,x):
    print(f'Starting task: {x}...')
    f= open('mapreducefinal/part-%s-final.csv'%x, 'w', encoding='UTF8',newline='') 
    csvs_reduce.append('mapreducefinal/part-%s-final.csv'%x)
    writer = csv.writer(f)
    header=["key","value"]
    writer.writerow(header)
    keyval=[key,len(value.split(','))]
    writer.writerow(keyval)
    print(f'Finished task: {x}...') 
    return f.name

def thread_reduce_func(csvs_map ,Function=reduce_function):
    df = load_data(csvs_map)
    key,value = df['key'],df['value']
    # create threads
    threads = []
    x = 1
    for i in range(df.shape[0]):
        thread = Thread(target=Function, args=(key[i],value[i],x))
        threads.append(thread)
        thread.start()
        x+=1

    # wait for the threads to complete
    for thread in threads:
        thread.join()
    print('MapReduce Completed')

class MapReduceEngine():
    def execute(self,input_data, map_function, reduce_function,threshold):
        thread_map_func(input_data,map_function,threshold)
        thread_reduce_func(csvs_map ,Function = reduce_function) 
# p1 = MapReduceEngine()
# p1.execute(input_data, map_function, reduce_function)

In [10]:
p1 = MapReduceEngine()
p1.execute(input_data, map_function, reduce_function,threshold=300)

['mapreducetemp/batch-tmp-1.csv', 'mapreducetemp/batch-tmp-2.csv', 'mapreducetemp/batch-tmp-3.csv', 'mapreducetemp/batch-tmp-4.csv', 'mapreducetemp/batch-tmp-5.csv', 'mapreducetemp/batch-tmp-6.csv', 'mapreducetemp/batch-tmp-7.csv', 'mapreducetemp/batch-tmp-8.csv', 'mapreducetemp/batch-tmp-9.csv', 'mapreducetemp/batch-tmp-10.csv', 'mapreducetemp/batch-tmp-11.csv', 'mapreducetemp/batch-tmp-12.csv', 'mapreducetemp/batch-tmp-13.csv', 'mapreducetemp/batch-tmp-14.csv', 'mapreducetemp/batch-tmp-15.csv', 'mapreducetemp/batch-tmp-16.csv', 'mapreducetemp/batch-tmp-17.csv', 'mapreducetemp/batch-tmp-18.csv', 'mapreducetemp/batch-tmp-19.csv']
Starting task: 1...
Starting task: 2...
Starting task: 3...
Starting task: 4...
Starting task: 5...
Starting task: 6...
Starting task: 7...Finished task: 1...Starting task: 8...
Finished task: 2...

Finished task: 3...Starting task: 9...
Finished task: 4...
Finished task: 5...Starting task: 10...
Finished task: 7...


Starting task: 11...
Finished task: 8...

